In [3]:
import glob
import zipfile 
import os
import xml.etree.ElementTree as ET
import collections
import csv
import random
from bs4 import BeautifulSoup
import progressbar
import numpy as np
from collections import deque

In [4]:
paths_common = ['GESTION/NOJO',
                'GESTION/TETIER_R4',
                'GESTION/NOM_HTML',
                'DONNEES/IDENT/NOM',
                'DONNEES/IDENT/PRM',
                'DONNEES/IDENT/ADRESSE',
                'DONNEES/IDENT/CP',
                'DONNEES/IDENT/VILLE',
                'DONNEES/IDENT/MEL',
                'DONNEES/IDENT/TEL',
                'DONNEES/OBJET/OBJET_COMPLET'
                ]

#give_me_the_tag = ['DONNEES/INFO',
                   #'DONNEES/TYPE_ACTIVITE_ORG']

paths_take_all = ['DONNEES/PROCEDURES/LOTS',
                  'DONNEES/CLASSES',
                  'DONNEES/DESCRIPTEURS',
                  'DONNEES/CARACTERISTIQUES']

jump_a_line = ['DONNEES/PROCEDURES/LOTS']

In [5]:
def path2tag(path):
    
    pavel = ''
    
    for i in reversed(path):
        if i != '/':
            pavel += i
        if i == '/':
            break
    return pavel[::-1]

In [6]:
path

NameError: name 'path' is not defined

In [7]:
def xml_parser(paths_common, paths_take_all, file):
    pairs=[]
    
    for path in paths_common:
        element = file.find(path)
        if element is not None:
            if element.text is not None:
                pairs.append([element.tag, element.text])
            else:
                pairs.append([element.tag, ''])
        elif element is None:
            pairs.append([path2tag(path), ''])
    """
    for path in give_me_the_tag:
        element = file.find(path)
        pavel = ''
        if element is not None:
            for child in element.getchildren():
                if child is not None:
                    pavel += child.tag
            pairs.append([path2tag(path), pavel])
        elif element is None:
            pairs.append([path2tag(path), ''])
    """     
    
    for path in paths_take_all:
        element = file.find(path)
        
        if element is not None:
            reslist = list(element.iter())
            if path == paths_take_all[0]:
                result = ' '.join(['{} : {}'.format(element.tag, element.text) for element in reslist if element.text is not None])
            else:    
                result = ' '.join([element.text for element in reslist if element.text is not None])
            pairs.append([element.tag, result])
        elif element is None:
            pairs.append([path2tag(path), ''])
            
    return pairs 


In [8]:
def write_to_csv(big_pair_list, file_name):
    bar3 = pbar(len(big_pair_list))
    big= deque()
    with open('{}.csv'.format(file_name), 'w') as file:
        spamwriter = csv.writer(file, delimiter=',', quotechar=' ',
                            quoting=csv.QUOTE_ALL)
        for pair_list in big_pair_list:
            bar3.update(big_pair_list.index(pair_list))
            
            #print(pair_list)
            tags = []
            text = []
            
            for pair in pair_list:
                tags.append(pair[0]) #nul car vraiment trop lent,
                text.append(pair[1]) #tout transformer en np.array?
            spamwriter.writerow(tags)
            spamwriter.writerow(text)
            #big.append(tags)
            #big.append(text)
        np.savetxt(file, np.asarray(big), delimiter=",", fmt='%s')
            

In [9]:
def pbar(maxlen):
    
    bar = progressbar.ProgressBar(maxval=maxlen, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(), ' (', progressbar.ETA(), ') '])
    
    bar.start()
    
    return bar

In [10]:
def write_years2csv(start, end):
    
    years = [start + i for i in range(end - start + 1)]
    
    for i in range(len(years)):
        #paths = glob.glob('{}/mpc/*.xml'.format(years[i]))
        paths = glob.glob('rand/*.xml')
        annonces = []
    
        bar = pbar(len(paths))
        
    
        for path in paths:
            bar.update(paths.index(path))
            file = open(path, encoding='ISO-8859-1')
            apple = file.read()
            
            apple = apple.replace('&apos;', "'")
            apple = apple.replace('&deg;', '°')
            apple = apple.replace('&quot;', '"')
            apple = apple.replace('\t', '')
            apple = apple.replace(',', '')
            apple = apple.replace('\n', '')
            apple = apple.replace('é', 'e')
            apple = apple.replace('ç', 'c')
            apple = apple.replace('é','e')
            apple = apple.replace('à', 'a')
            apple = apple.replace('ù', 'u')
            apple = apple.replace('ê','e') #il y a forcément un moyen plus simple
            tree = ET.fromstring(apple)
            a = tree.getchildren()
    
            annonces.append(a)
    
        p = []
        bar2 = pbar(len(annonces))
        for annonce in annonces:
            bar2.update(annonces.index(annonce))
            for j in range(len(annonce)):
                extracted_annonce = xml_parser(paths_common, paths_take_all, annonce[j])
                p.append(extracted_annonce)
        write_to_csv(p, years[i])

    

In [11]:
write_years2csv(2013, 2013)

[===================================================== ]  99% (ETA:   0:00:00) 

In [14]:
paths = glob.glob('rand/*.xml')
file = open(random.choice(paths), encoding='ISO-8859-1')
apple = file.read()
apple = apple.replace('&apos;', "'")
apple = apple.replace('&deg;', '°')
apple = apple.replace('&quot;', '"')
tree = ET.fromstring(apple)
a = tree.getchildren()


In [20]:
for key in tree.keys():
    print(tree.get(key))

20130110
11 juin 2013
BOMP C


In [23]:
a[0].find('GESTION/NOJO').text

'13-102604'

In [148]:
classes.getchildren()

[<Element 'NUM_LOT' at 0x7fa8e779a368>,
 <Element 'NUM_LOT' at 0x7fa8e779a098>,
 <Element 'DESC' at 0x7fa8e779a778>,
 <Element 'NOM' at 0x7fa8e779a4f8>,
 <Element 'ADRESSE' at 0x7fa8e779a6d8>,
 <Element 'CP' at 0x7fa8e779a638>,
 <Element 'VILLE' at 0x7fa8e779a688>,
 <Element 'DATE_ATT_MARCHE' at 0x7fa8e779a908>,
 <Element 'NB_OFFRES' at 0x7fa8e779ab38>,
 <Element 'NUM_LOT' at 0x7fa8f38b5728>,
 <Element 'DESC' at 0x7fa8f38b5048>,
 <Element 'NOM' at 0x7fa8f38b56d8>,
 <Element 'ADRESSE' at 0x7fa8f38b5e08>,
 <Element 'CP' at 0x7fa8f38b5598>,
 <Element 'VILLE' at 0x7fa8f38b5868>,
 <Element 'DATE_ATT_MARCHE' at 0x7fa8f38b57c8>,
 <Element 'NB_OFFRES' at 0x7fa8f38b5778>,
 <Element 'NUM_LOT' at 0x7fa8f38b58b8>,
 <Element 'DESC' at 0x7fa8f38b5a98>,
 <Element 'NOM' at 0x7fa8f38b5c28>,
 <Element 'ADRESSE' at 0x7fa8f38b5958>,
 <Element 'CP' at 0x7fa8f38b59a8>,
 <Element 'VILLE' at 0x7fa8f38b5818>,
 <Element 'DATE_ATT_MARCHE' at 0x7fa8f38b5908>,
 <Element 'NB_OFFRES' at 0x7fa8f38b5138>]

In [156]:
reslist = list(classes.iter())
result = ' '.join(["{}: {}".format(element.tag, element.text) for element in reslist if element.text is not None])

In [189]:
''.join(["hey"])

'hey'